In [ ]:
import pandas as pd
from googleapiclient.discovery import build
import random
import time

# Define API key and YouTube API service
api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxx'
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_id(youtube, channel_username):
    try:
        # Search for the channel by username (which is part of the URL)
        request = youtube.channels().list(
            forUsername=channel_username,
            part='id'
        )
        response = request.execute()

        if 'items' in response and len(response['items']) > 0:
            return response['items'][0]['id']
        else:
            return None
    except Exception as e:
        print(f"Error fetching channel ID for {channel_username}: {e}")
        return None

def check_youtube_channel_ads(channel_id, start_year, end_year, max_ads=100):
    start_date = f'{start_year}-01-01T00:00:00Z'
    end_date = f'{end_year}-12-31T23:59:59Z'
    ads_info = []
    next_page_token = None

    try:
        while True:
            request = youtube.search().list(
                channelId=channel_id,  # Filter by the specific channel ID
                part='snippet',
                publishedAfter=start_date,
                publishedBefore=end_date,
                type='video',
                maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()

            if 'items' in response:
                for item in response['items']:
                    video_date = item['snippet']['publishedAt']
                    video_url = f"https://www.youtube.com/watch?v={item['id']['videoId']}"

                    # Append the relevant data
                    ads_info.append({
                        'ChannelID': channel_id,
                        'Date': video_date,
                        'AdLink': video_url
                    })

            # Check if there's another page of results
            next_page_token = response.get('nextPageToken')
            if not next_page_token or len(ads_info) >= max_ads:
                break

            time.sleep(1)  # Pause to avoid hitting rate limits

        # Randomly select up to max_ads ads from the list
        if len(ads_info) > max_ads:
            ads_info = random.sample(ads_info, max_ads)

        return ads_info
    except Exception as e:
        print(f"An error occurred while fetching data for channel {channel_id}: {e}")
        return []

def main():
    # Load company names from CSV
    df = pd.read_csv('/content/50comp.csv')  # Replace with your CSV file path
    company_names = df['Name'].tolist()  # Adjust column name as per your CSV structure
    all_ads_info = []

    for company in company_names:
        # Construct the YouTube username (part of the URL)
        youtube_username = company  # The company name is used directly as part of the channel URL
        channel_id = get_channel_id(youtube, youtube_username)

        if channel_id:
            # Fetch the channel's ads
            ads_info = check_youtube_channel_ads(channel_id, 2021, 2024, max_ads=100)
            all_ads_info.extend(ads_info)
            print(f"Company: {company}, Number of Ads Found: {len(ads_info)}")
        else:
            print(f"Channel not found for {company}.")

        time.sleep(1)  # Sleep to avoid hitting rate limits

    # Save results to a new CSV file
    if all_ads_info:  # Check if there's any data to save
        ads_df = pd.DataFrame(all_ads_info)
        ads_df.to_csv('youtube_channel_ads_info.csv', index=False)
        print("Results saved to youtube_channel_ads_info.csv")
    else:
        print("No advertisements found for any company in the specified period.")
